### Pivoting between Logs

Pivots provide a mechanism for going from a conn.log based event to the corresponding network log events, and vice-versa.

In [1]:
from datetime import datetime, timedelta

from dynamite_sdk.search import Search
from dynamite_sdk.search import ConnectionEventToNetworkEventsPivot, NetworkEventToConnectionEventPivot

In [2]:
start = datetime.now() - timedelta(minutes=60)
end = datetime.now()

#### Find the corresponding HTTP logs (`http.log`) corresponding to a set of HTTP connection events (`conn.log`)

In [3]:
search = Search('conn', as_dataframe=False)

In [4]:
search.execute_query(start, end, search_filter='zeek.service: http')

#### Iterate through each of the matching `conn.log` entries, and locate their corresponding network protocol logs.

Since we filtered by HTTP services in this case, corresponding logs will be from `http.log`.

In [5]:
for conn_event in search.events:
    # Setup our Pivot object around a conn.log event
    net_log_pivot = ConnectionEventToNetworkEventsPivot(conn_event)
    # Execute our pivot
    net_log_pivot.execute_pivot()
    # There can be multiple network logs for a given conn.log entry, so we must iterate through them
    for http_log in net_log_pivot.events:
        print(http_log.event_time, http_log.method, http_log.uri)

2019-11-01 13:01:44.499000+00:00 GET /latest/meta-data/iam/security-credentials
2019-11-01 12:53:10.696000+00:00 GET /latest/meta-data/network/interfaces/macs/02:d6:a8:e0:bb:38/local-ipv4s
2019-11-01 12:23:52.956000+00:00 GET /latest/meta-data/network/interfaces/macs/02:d6:a8:e0:bb:38/local-ipv4s


#### Find the corresponding Connections (`conn.log`) corresponding to a set of HTTP logs (`http.log`)

In [6]:
search = Search('http', as_dataframe=False)

In [10]:
search.execute_query(start, end)

In [14]:
for http_event in search.events:
    # Setup our Pivot object around an http.log event
    conn_log_pivot = NetworkEventToConnectionEventPivot(http_event)
    # Execute our pivot
    conn_log_pivot.execute_pivot()
    # Iterate through results
    for conn_log in conn_log_pivot.events:
        print(conn_log.uid, conn_log.event_time, conn_log.source_ip_address, conn_log.destination_ip_address)

Cc1FXLB7Ff90OwE41 2019-11-01 13:01:44.499000+00:00 192.168.53.158 169.254.169.254
Ce5QS38GBTd9MuDud 2019-11-01 12:53:10.696000+00:00 192.168.53.158 169.254.169.254
C8Y4wX2LwuouQOKesa 2019-11-01 12:23:52.955000+00:00 192.168.53.158 169.254.169.254
